In [1]:
## A generic code to construct your own crosswalk, from two shapefiles

import pandas as pd
import geopandas as gpd
import os

In [4]:
## defining variables - change the things in ALL_CAPS
reporting_path = '/Users/joseaevan5/Documents/gis3/sg_boundary'
reporting_fname = 'sg_boundary_2020.shp'
reporting_geoid = 'Name'

reference_path = '/Users/joseaevan5/Documents/gis3/pa'
reference_fname = 'pa_4326.shp'
reference_geoid = 'OBJECTID'

output_path = 'crosswalk'
output_fname = 'crosswalk_output'

# A note: the reporting and reference geo_ids should be different.

## read in starting shapefile
os.chdir(reporting_path)
shp_reporting = gpd.GeoDataFrame.from_file(reporting_fname)
shp_reporting['area_base'] = shp_reporting.area

## read in ending shapefile
os.chdir(reference_path)
shp_reference = gpd.GeoDataFrame.from_file(reference_fname)

## intersecting the file
intersect = gpd.overlay(shp_reporting, shp_reference, how = 'intersection')
intersect['area'] = intersect.area

## computing weights
intersect['weight'] = intersect['area'] / intersect['area_base']

## renormalizing weights - this isn't necesary, but without it, if the shapefiles do not perfectly line up where they should, you may lose small fractions of area here and there
reweight = intersect.groupby(reporting_geoid)['weight'].sum().reset_index()
reweight['new_weight'] = reweight['weight']
reweight = reweight.drop('weight', axis = 1)

intersect = intersect.merge(reweight, left_on = reporting_geoid, right_on = reporting_geoid)
intersect['weight'] = intersect['weight'] / intersect['new_weight']

intersect = intersect.drop('new_weight', axis =1)


## keeping only relevant columns - again isn't necessary, but will help trim down the size of the crosswalk at the end
output = intersect[[reporting_geoid, reference_geoid, 'weight']]


## saving output
os.chdir(output_path)
output.to_csv(output_fname, index = False)



# A final note - if you have OSErrors, there is something wrong with your environment and shapely.
# A stated fix can be found here: https://github.com/conda-forge/shapely-feedstock/issues/64
# Thank you to Daniel Bischof for pointing out this potential problem and finding the solution.

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



GEOSException: Unknown exception thrown